## drawPostProc
### Quick script to test and plot individual FCCD post procs

In [7]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import os
import fnmatch
import h5py
%matplotlib widget

In [2]:
#Here I just wanted to check the output of analysis_DL_top.py worked as expected for TL implementation
#Compare "fast" code with normal


hdf5_path = "/lfs/l1/legend/users/aalexander/hdf5_output/processed/"

DLF100 =  hdf5_path+"processed_detector_IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm.hdf5"
DLF50 = hdf5_path+"valentina_script/processed_detector_IC160A_ba_top_81mmNEW8_01_g_FCCD0.74mm_DLF0.5.hdf5"
DLF50_fast = hdf5_path+"valentina_script/processed_detector_IC160A_ba_top_81mmNEW8_01_g_FCCD0.74mm_DLF0.5_fast.hdf5"


df_MC_DLF100 =  pd.read_hdf(DLF100, key="procdf")
energy_MC_DLF100 = df_MC_DLF100['energy']*1000

df_MC_DLF50 =  pd.read_hdf(DLF50, key="procdf")
energy_MC_DLF50 = df_MC_DLF50['energy']#*1000

df_MC_DLF50_fast =  pd.read_hdf(DLF50_fast, key="procdf")
energy_MC_DLF50_fast = df_MC_DLF50_fast['energy']#*1000

fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)
plt.hist(energy_MC_DLF100, bins = bins, label ='MC FCCD:0.74mm, DLF=1', histtype = 'step', linewidth = '0.35')
plt.hist(energy_MC_DLF50, bins = bins, label ='MC FCCD:0.74mm, DLF=0.5', histtype = 'step', linewidth = '0.35')
plt.hist(energy_MC_DLF50_fast, bins = bins, label ='MC FCCD:0.74mm, DLF=0.5 FASt', histtype = 'step', linewidth = '0.35')
plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
#load data

sys.path.append('../../data/')
from Ba133_data_AV_analysis import * 

#load data, cuts and calibration
data_path = "/lfs/l1/legend/detector_char/enr/hades/char_data/I02160A/tier2/ba_HS4_top_dlt/pygama/v00.00/"
with open("../../data/detectors/I02160A/calibration_coef.json") as json_file:
    calibration_coefs = json.load(json_file)
    m = calibration_coefs['m']
    m_err = calibration_coefs['m_err']
    c = calibration_coefs['c']
    c_err = calibration_coefs['c_err']
    
df_data = read_all_dsp_h5(data_path, cuts=False)
e_ftp_data = df_data['e_ftp']
energy_data = (e_ftp_data-c)/m

passed_cuts = json.load(open('/lfs/l1/legend/users/aalexander/large_files/cuts/I02160A_ba_top_passed_cuts_data.json','r')) #passed cuts
df_data_cuts = read_all_dsp_h5(data_path, cuts=True, passed_cuts=passed_cuts)
e_ftp_data_cuts = df_data_cuts['e_ftp']
energy_data_cuts = (e_ftp_data_cuts-c)/m


In [4]:
#for scaling MC
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0744 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0744)

3.1255920187062203


In [5]:
#compare slow and fast codes - they give the same output

hdf5_path = "/lfs/l1/legend/users/aalexander/hdf5_output/processed/"

fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.1')

DLF_list = [0.0, 0.5, 1.0]
for DLF in DLF_list:
    
    DLFi = hdf5_path+"valentina_script/processed_detector_IC160A_ba_top_81mmNEW8_01_g_FCCD0.74mm_DLF"+str(DLF)+"_fast.hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD0744)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.74mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.1')
    
    
    #compare with "slow" - THEY GIVE THE SAME OUTPUT!
    DLFi = hdf5_path+"valentina_script/processed_detector_IC160A_ba_top_81mmNEW8_01_g_FCCD0.74mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD0744)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.74mm, DLF='+str(DLF)+' (scaled, SLOW)', histtype = 'step', linewidth = '0.1')
    
  
plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")
    
    
    
    
    
    
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#Compare all DLFs, fast code, old simulation

hdf5_path = "/lfs/l1/legend/users/aalexander/hdf5_output/processed/"

fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.1')

DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]

for DLF in DLF_list:
    
    DLFi = hdf5_path+"valentina_script/processed_detector_IC160A_ba_top_81mmNEW8_01_g_FCCD0.74mm_DLF"+str(DLF)+"_fast.hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD0744)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.74mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.1')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#Compare all DLFs, fast code, NEW simulation
## with REFITed BEST FIT FCCD ON NEW SIMULATION
# NO CUTS

hdf5_path = "/lfs/l1/legend/users/aalexander/hdf5_output/raw_MC_combined/processed/"
MC_file_id = "IC160A-BA133-uncollimated-top-run0003-81z-newgeometry_g"


#______SCALED MC TO DLF 1_____
fig, ax = plt.subplots()
binwidth = 0.5
bins = np.arange(0, 450, binwidth)
plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')
with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.71mm_DLF1.0_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD071_DLF1 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD071_DLF1)
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.71mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD071_DLF1)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.71mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")

#______SCALED MC TO DLF 0_____
fig, ax = plt.subplots()
binwidth = 0.5
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')

with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.71mm_DLF0.0_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD071_DLF0 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD071_DLF0)
    
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.71mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD071_DLF0)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.71mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")


#______SCALED MC TO EACH DLF_____
fig, ax = plt.subplots()
binwidth = 0.25
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')
    
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.71mm_DLF'+str(DLF)+'_dlt_observables.json') as json_file:
        dlt_observables = json.load(json_file)
        R_simdata_356_FCCD071_DLFi = dlt_observables['R_simdata_356_counts']
        print(R_simdata_356_FCCD071_DLFi)
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.71mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD071_DLFi)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.71mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2.0719888070911536


/home/extern/aalexander/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.685379649151359


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.685379649151359
1.8783517994425867
1.973098178394095
2.0304665869897716
2.0719888070911536


In [16]:
#Compare all DLFs, fast code, NEW simulation
## with REFITed BEST FIT FCCD ON NEW SIMULATION
# CUTS

hdf5_path = "/lfs/l1/legend/users/aalexander/hdf5_output/raw_MC_combined/processed/"
MC_file_id = "IC160A-BA133-uncollimated-top-run0003-81z-newgeometry_g"


#______SCALED MC TO DLF 1_____
fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)
plt.hist(energy_data_cuts, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')
with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.67mm_DLF1.0_dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD067_DLF1 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD067_DLF1)
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.67mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD067_DLF1)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.67mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")

#______SCALED MC TO DLF 0_____
fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data_cuts, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')

with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.67mm_DLF0.0_dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD067_DLF0 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD067_DLF0)
    
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.67mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD067_DLF0)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.67mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")


#______SCALED MC TO EACH DLF_____
fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data_cuts, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')
    
DLF_list = [0.0, 0.25, 0.5, 0.75, 1.0]
#DLF_list = [0.0]
for DLF in DLF_list:
    
    with open('../detectors/I02160A/'+MC_file_id+'_FCCD0.67mm_DLF'+str(DLF)+'_dlt_observables_cuts.json') as json_file:
        dlt_observables = json.load(json_file)
        R_simdata_356_FCCD067_DLFi = dlt_observables['R_simdata_356_counts']
        print(R_simdata_356_FCCD067_DLFi)
    
    DLFi = hdf5_path+"processed_detector_"+MC_file_id+"_FCCD0.67mm_DLF"+str(DLF)+".hdf5"
    df_MC_DLFi =  pd.read_hdf(DLFi, key="procdf")
    energy_MC_DLFi = df_MC_DLFi['energy']#*1000
    plt.hist(energy_MC_DLFi, bins = bins, weights=(1/R_simdata_356_FCCD067_DLFi)*np.ones_like(energy_MC_DLFi), label ='MC FCCD:0.67mm, DLF='+str(DLF)+' (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2.8541815429064172


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2.3294857323648355


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2.3294857323648355
2.589092605980488
2.7184227195309436
2.796496366885824
2.8541815429064172


In [20]:
# check FCCD0 for new simulation


def f_smear(x):
    a=0.35 #0.27  #new values from ge-proc upgrades  #old value: 0.35
    b=1.99e-3   #2.08e-3                                  #old value: 1.99e-3
    return np.sqrt(a+b*x)


MC_raw = "/lfs/l1/legend/users/aalexander/hdf5_output/raw_MC_combined/raw-IC160A-BA133-uncollimated-top-run0003-81z-newgeometry.hdf5"


g4sdf = pd.read_hdf(MC_raw, key="procdf")

detector_hits = g4sdf.loc[(g4sdf.Edep>0)&(g4sdf.volID==1)]
procdf = pd.DataFrame(detector_hits.groupby(['event','volID','iRep'], as_index=False)['Edep'].sum())
procdf = procdf.rename(columns={'iRep':'detID', 'Edep':'energy'})
procdf = procdf[procdf.energy!=0]    
procdf['energy']=procdf['energy']*1000+(f_smear(procdf['energy']*1000))/2.355*np.random.randn(len(procdf['energy']))
    
energy_MC = procdf['energy']

fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)

plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')
plt.hist(energy_MC, bins = bins, weights=(1/R_simdata_356_FCCD071_DLF1)*np.ones_like(energy_MC), label ='MC FCCD:0mm, (scaled)', histtype = 'step', linewidth = '0.2')

plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
#Compare each individual raw MC file for raw-IC160A-BA133-uncollimated-top-run0003-81z-newgeometry.hdf5"
#results - they are the same, error must lie in combine_simulations.py

MC_dir = "/lfs/l1/legend/detector_char/enr/hades/simulations/legend-g4simple-simulation/IC-legend/IC160A/Ba133/uncollimated/top/"
MC_id = "raw-IC160A-BA133-uncollimated-top-run0003-81z-newgeometry"


fig, ax = plt.subplots()
binwidth = 0.15
bins = np.arange(0, 450, binwidth)
#plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.2')

#read in each hdf5 file
files = os.listdir(MC_dir)
files = fnmatch.filter(files, "*.hdf5")
for index, file_name in enumerate(files):

    print("file: ", str(file_name))
    g4sfile = h5py.File(MC_dir+file_name, 'r')
    g4sntuple = g4sfile['default_ntuples']['g4sntuple']
    g4sdf = pd.DataFrame(np.array(g4sntuple), columns=['event'])

    # # build a pandas DataFrame from the hdf5 datasets we will use
    g4sdf = pd.DataFrame(np.array(g4sntuple['event']['pages']), columns=['event'])
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['step']['pages']), columns=['step']),lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['Edep']['pages']), columns=['Edep']),lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['volID']['pages']),columns=['volID']), lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['iRep']['pages']),columns=['iRep']), lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['x']['pages']),columns=['x']), lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['y']['pages']),columns=['y']), lsuffix = '_caller', rsuffix = '_other')
    g4sdf = g4sdf.join(pd.DataFrame(np.array(g4sntuple['z']['pages']),columns=['z']), lsuffix = '_caller', rsuffix = '_other')

    
    detector_hits = g4sdf.loc[(g4sdf.Edep>0)&(g4sdf.volID==1)]
    procdf = pd.DataFrame(detector_hits.groupby(['event','volID','iRep'], as_index=False)['Edep'].sum())
    procdf = procdf.rename(columns={'iRep':'detID', 'Edep':'energy'})
    procdf = procdf[procdf.energy!=0] 
    
    print(procdf)
    
    energy_MC = (procdf["energy"])*1000
    
    file_no = file_name.strip(MC_id)
    file_no = file_no.strip(".hdf5")
    print("file_no: ", file_no)
    plt.hist(energy_MC, bins = bins, label ='file '+file_no, histtype = 'step', linewidth = '0.2')


plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc="lower left")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

file:  raw-IC160A-BA133-uncollimated-top-run0003-81z-newgeometry-01.hdf5
          event  volID  detID    energy
0             6      1      0  0.302851
1            16      1      0  0.029979
2            24      1      0  0.091357
3            46      1      0  0.136133
4            72      1      0  0.080998
...         ...    ...    ...       ...
673259  9999918      1      0  0.063974
673260  9999922      1      0  0.356013
673261  9999925      1      0  0.073279
673262  9999974      1      0  0.074485
673263  9999978      1      0  0.030622

[673264 rows x 4 columns]
file_no:  
file:  raw-IC160A-BA133-uncollimated-top-run0003-81z-newgeometry-05.hdf5
          event  volID  detID    energy
0             2      1      0  0.032272
1            14      1      0  0.080998
2            16      1      0  0.062707
3            24      1      0  0.029505
4            48      1      0  0.030979
...         ...    ...    ...       ...
674433  9999893      1      0  0.045762
674434  9999918 